# Dump Bert embedding for KB and dialog utterances

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

model_name = "bert-base-uncased" # pretrained model name
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
%env HOME=/content/drive/MyDrive/
%cd ~/
!mkdir -p dstc9_bertemb
%cd dstc9_bertemb

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive
/content/drive/MyDrive/dstc9_bertemb


In [ ]:
!ls -lrt ../Research/dstc10_debug
!mkdir glue_model
!cd glue_model; tar xvf ~/Research/dstc10_debug/glue.tar

total 4500390
-rw------- 1 root root 1099048960 Jun 22 14:38 ktd-fix-baseline.tar
drwx------ 2 root root       4096 Jun 22 14:41 ktd-fix-baseline
-rw------- 1 root root  549519360 Jun 23 01:50 ktd-lastturn-baseline.tar
drwx------ 2 root root       4096 Jun 23 01:52 ktd-lastturn-baseline
-rw------- 1 root root  549529600 Jun 23 11:45 ktd-baseline-ckp31220.tar
drwx------ 2 root root       4096 Jun 24 09:15 ktd-fix-baseline-rerun
-rw------- 1 root root  549519360 Jun 25 05:49 ktd-lastturn-ckp26760.tar
drwx------ 2 root root       4096 Jun 25 05:55 ktd-lastrun-rerun
-rw------- 1 root root  590960640 Jul  5 07:19 alex10.tar
-rw------- 1 root root  392970240 Jul  5 07:23 alex9.tar
drwx------ 2 root root       4096 Jul 21 05:45 runs1
-rw------- 1 root root  438835200 Jul 23 07:24 mlm.tar
-rw------- 1 root root  437995520 Jul 28 09:56 glue.tar
config.json
pytorch_model.bin


In [ ]:
!ls -lt glue_model

total 427726
-rw------- 1 root root 437990081 Jul 27 13:10 pytorch_model.bin
-rw------- 1 root root       699 Jul 27 13:10 config.json


In [ ]:
!git clone https://github.com/alexa/alexa-with-dstc9-track1-dataset.git

fatal: destination path 'alexa-with-dstc9-track1-dataset' already exists and is not an empty directory.


In [ ]:
!ls alexa-with-dstc9-track1-dataset/data

kb_eval_query.txt  kb_query.txt  kb_wtemplates.json  output_schema.json  train
kb.pkl		   kb_vecs.npy	 knowledge.json      README.md		 val


In [ ]:
dataroot = 'alexa-with-dstc9-track1-dataset/data'
dataroot_eval = 'alexa-with-dstc9-track1-dataset/data_eval'

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 8.0 MB/s 
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
     |████████████████████████████████| 636 kB 60.4 MB/s 
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Deep Pre-trained model

In [ ]:
!pwd

/content/drive/My Drive/dstc9_bertemb


In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForSequenceClassification

model_name = "bert-base-uncased"

config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
#model = AutoModel.from_pretrained("./glue_model")
model = AutoModelForSequenceClassification.from_pretrained("./glue_model")

# Set device
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
state_dict = model.state_dict()
state_dict["bert.embeddings.word_embeddings.weight"]

tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       device='cuda:0')

In [ ]:
#tokenizer(["I'd like to have some Chinese food.", "hey"], add_special_tokens = True, return_tensors="pt", padding=True)
#sentence = "There are many options to choose from. Do you have a type of food in mind[SEP][MASK] like to have some Chinese food"
#sentence = "The hotel is in the south area[SEP]Do they have help for disabled [MASK]"
sentence = "What parking is offered?"
body = "It offers off street, street, and validated parking."
sentence2 = "What parking is offered?"
body2 = "It offers off street, street, and validated parking."
#result = tokenizer([(sentence, body)], return_tensors="pt", add_special_tokens=True, return_special_tokens_mask=False, padding=True)
result = tokenizer([(sentence, body), (sentence2, body2)], return_tensors="pt")
result.to(device)

#print(result.input_ids.size())

#context = tokenizer.tokenize(sentence)
#print(context)

#index = context.index(tokenizer.mask_token) - context.index(tokenizer.sep_token) -1
#print(index)
#lm_labels = [-100] * (index + context.index(tokenizer.sep_token) + 1) + [index] + [-100] * (len(context) - (context.index(tokenizer.mask_token) + 1))
#print(context)
#print(lm_labels)
#
#print(result)
#result.index(tokenizer.mask_token)
#result["input_ids"]

{'input_ids': tensor([[ 101, 2054, 5581, 2003, 3253, 1029,  102, 2009, 4107, 2125, 2395, 1010,
         2395, 1010, 1998, 9398, 4383, 5581, 1012,  102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
model.eval()
outputs = model(**result)
print(outputs.keys())
print(outputs)
predictions = outputs.logits.argmax(dim=-1)
print(predictions)

odict_keys(['logits'])
SequenceClassifierOutput(loss=None, logits=tensor([[ 6.0067, -4.4582]], device='cuda:0', grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)
tensor([0], device='cuda:0')


In [ ]:
print(tokenizer.pad_token)
print(tokenizer.pad_token_id)

[PAD]
0


In [ ]:
!pwd

/content/drive/My Drive/dstc9_bertemb


## Dataset

Read DSTC9 dataset in json format.

In [ ]:
#%%writefile user_utterance_iter.py

import os
import json
from torch.utils.data import IterableDataset

class UserUtteranceIterator(IterableDataset):
  def __init__(self, dataroot, dataset, tokenizer):
    path = os.path.join(os.path.abspath(dataroot))
    
    if dataset not in ['train', 'val', 'test']:
      raise ValueError('Wrong dataset name: %s' % (dataset))
      
    self.user_utterances = {}

    logs_file = os.path.join(path, dataset, 'logs.json')
    with open(logs_file, 'r') as f:
      self.logs = json.load(f)
      for i, log in enumerate(self.logs):
        for j, turn in enumerate(log):
          query = turn["text"]

          # session i, turn j
          qid = "{}_{}".format(i, j)
          if turn["speaker"] == "U":
            if query not in self.user_utterances:
              self.user_utterances[query] = [qid]
            else:
              self.user_utterances[query].append(qid)

    self.tokenizer = tokenizer

  def __iter__(self):
    for q in self.user_utterances:
      yield q

  def __len__(self):
    return len(self.user_utterances)

  def save(self, out_file):
    with open(out_file, 'w') as f:
      for q in self.user_utterances:
        f.write("{}\n".format(q))

  def collate_fn(self, batch):
    """
    Received a batch of queries.
    Return: dictionary of inputs
    """
    output = self.tokenizer(batch, padding=True, add_special_tokens = True, return_tensors="pt")

    output_dict = {x: output[x].to(device) for x in output}
    output_dict["query"] = batch

    return output_dict

In [ ]:
from torch.utils.data import DataLoader
#from user_utterance_iter import UserUtteranceIterator

train_dataset = UserUtteranceIterator(dataroot, 'train', tokenizer)
valid_dataset = UserUtteranceIterator(dataroot, 'val', tokenizer)
test_dataset = UserUtteranceIterator(dataroot_eval, 'test', tokenizer)

train_dataloader = DataLoader(
  train_dataset,
  batch_size=batch_size,
  collate_fn=train_dataset.collate_fn
)

valid_dataloader = DataLoader(
  valid_dataset,
  batch_size=batch_size,
  collate_fn=valid_dataset.collate_fn
)

test_dataloader = DataLoader(
  test_dataset,
  batch_size=batch_size,
  collate_fn=test_dataset.collate_fn
)

In [ ]:
print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))

64250
9320
11435


In [ ]:
train_dataset.save(os.path.join(dataroot, 'train', 'train_query.txt'))
valid_dataset.save(os.path.join(dataroot, 'val', 'valid_query.txt'))
test_dataset.save(os.path.join(dataroot_eval, 'test', 'test_query.txt'))

In [ ]:
# check the minibatch of samples
for i, batch in enumerate(train_dataloader):
  if i >= 2:
    break
  print(batch)

{'input_ids': tensor([[ 101, 2559, 2005,  ...,    0,    0,    0],
        [ 101, 1045, 1005,  ...,    0,    0,    0],
        [ 101, 1045, 2572,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2036,  ...,    0,    0,    0],
        [ 101, 2097, 2045,  ...,    0,    0,    0],
        [ 101, 1045, 2074,  ..., 2022, 1029,  102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'query': ['Looking for a place to eat in the city center.', "I'd like to have some Chinese food.", 'I am looking for a moderately pric

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = stopwords.words('english')
print(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /content/drive/MyDrive/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where'

In [ ]:
# import these modules
#from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
  
#ps = PorterStemmer()
ps = SnowballStemmer("english", ignore_stopwords=True)
 
# choose some words to be stemmed
words = ["facility", "facilities", "no,", "program", "programs", "programmer", "programming", "programmers", "concierge", "generously"]
 
for w in words:
    print(w, " : ", ps.stem(w))

facility  :  facil
facilities  :  facil
no,  :  no,
program  :  program
programs  :  program
programmer  :  programm
programming  :  program
programmers  :  programm
concierge  :  concierg
generously  :  generous


In [ ]:
s = "hey!"
s.translate(str.maketrans('', '', string.punctuation))

'hey'

## KB Dataset

In [ ]:
#%%writefile knowledge_db.py
import re
import os
import json
import string
from torch.utils.data import Dataset

import nltk
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

class KnowledgeDataset(Dataset):
  """
  Yield delexicalized title utterances
  """
  def __init__(self, dataroot, filename, tokenizer, title_field='title_template', body_field='body_template'):
    path = os.path.join(os.path.abspath(dataroot))
    
    kb_file = os.path.join(path, filename)
    with open(kb_file, 'r') as f:
      self.kb = json.load(f)

    self.ps = SnowballStemmer("english", ignore_stopwords=True)
    self.stopwords = stopwords.words('english')

    self.kid2doc = {}
    self.kb_utterances = {}
    self.kb_utterances_list = []

    # remove stopwords, sort remaining keywords on title
    self.kb_utterances1 = {}
    self.kb_utterances_list1 = []

    # remove stopwords, sort remaining keywords on body
    self.kb_utterances2 = {}
    self.kb_utterances_list2 = []

    self.entity_utterances = {}

    for domain in self.kb.keys():
      self.entity_utterances[domain] = []
      for i, entity in self.kb[domain].items():
        name = entity["name"]
        titles = []
        for j, qa, in entity["docs"].items():
          kid = "{}_{}_{}".format(domain, i, j)
          # Use title_template first. If not found, use the default title
          title = qa[title_field] if title_field in qa else qa["title"]
          body = qa[body_field] if body_field in qa else qa["body"]

          self.kid2doc[kid] = title + "\t" + body

          q = self.delexicalize(domain, name, title)
          q1 = self.norm_query(q)

          a = self.delexicalize(domain, name, body)
          a1 = self.norm_query(a)

          titles.append(q)

          if q not in self.kb_utterances:
            self.kb_utterances[q] = [kid]
            # only store uniq query
            self.kb_utterances_list.append(q)
          else:
            self.kb_utterances[q].append(kid)
            
          if q1 not in self.kb_utterances1:
            self.kb_utterances1[q1] = [kid]
            # only store uniq query
            self.kb_utterances_list1.append(q1)
          else:
            self.kb_utterances1[q1].append(kid)

          if a1 not in self.kb_utterances2:
            self.kb_utterances2[a1] = [kid]
            # only store uniq query
            self.kb_utterances_list2.append(a1)
          else:
            self.kb_utterances2[a1].append(kid)

        self.entity_utterances[domain].append(titles)
    self.tokenizer = tokenizer

    # merge equivalent entity docs in kb_utterances1 and kb_utterances2 
    self.doc2clusterIdx = {}
    self.doc_clusters = []
    num_cluster = 0

    # get the doc subsets
    for S in self.kb_utterances1.values():
      # check if any docid exists in current map
      c = None
      for docid in S:
        if docid in self.doc2clusterIdx:
          c = self.doc2clusterIdx[docid]
          break
      if c is None:
        # create new cluster idx c
        c = num_cluster
        num_cluster += 1
        self.doc_clusters.append(set({}))
      # add all docid into c
      for docid in S:
        self.doc2clusterIdx[docid] = c
        self.doc_clusters[c].add(docid)
        
    self.doc2clusterIdx2 = {}
    self.doc_clusters2 = []
    num_cluster2 = 0

    # get the doc subsets
    for S in self.kb_utterances2.values():
      # check if any docid exists in current map
      c = None
      for docid in S:
        if docid in self.doc2clusterIdx2:
          c = self.doc2clusterIdx2[docid]
          break
      if c is None:
        # create new cluster idx c
        c = num_cluster2
        num_cluster2 += 1
        self.doc_clusters2.append(set({}))
      # add all docid into c
      for docid in S:
        self.doc2clusterIdx2[docid] = c
        self.doc_clusters2[c].add(docid)

    # merge set 2
    for S in self.kb_utterances2.values():
      # check if any docid exists in current map
      cluster_set = {}
      for docid in S:
        if docid in self.doc2clusterIdx:
          c = self.doc2clusterIdx[docid]
          # keep count. We only trust count > 1 (multiple docids from body map to the same title cluster)
          if c in cluster_set:
            cluster_set[c] += 1
          else:
            cluster_set[c] = 1

      # now we should merge all docs in cluster_set into a new cluster index
      new_cluster = set({})
      for i in cluster_set.keys():
        # only trust merging clusters with count > 1
        if cluster_set[i] <= 1:
          continue
        for docid in self.doc_clusters[i]:
          # update docid to the new cluster idx
          self.doc2clusterIdx[docid] = num_cluster
          new_cluster.add(docid)
        # clear i
        self.doc_clusters[i] = set({})

      # discard the body cluster due to potential noise
      #for docid in S:
      #  self.doc2clusterIdx[docid] = num_cluster
      #  new_cluster.add(docid)

      self.doc_clusters.append(new_cluster)
      num_cluster += 1

    # remove singleton clusters
    self.doc_clusters = [c for c in self.doc_clusters if len(c) > 0]
    self.doc_clusters2 = [c for c in self.doc_clusters2 if len(c) > 0]

  def get_doc(self, kid):
    try:
      return self.kid2doc[kid]
    except:
      return "Not found"

  def norm_query(self, query):
    """
    remove stopwords, stem each word, sort the word order, remove punctuation
    """
    query = query.lower()
    # remove anything after
    #query = re.sub(r"this \w+", "", query)
    #query = re.sub(r"this restaurant", "", query)
    #query = re.sub(r"this attraction", "", query)
    #query = re.sub(r"this facility", "", query)
    #query = re.sub(r"this property", "", query)
    #query = re.sub(r"this location", "", query)
    #query = re.sub(r"this site", "", query)
    #query = re.sub(r"this place", "", query)
    #query = re.sub(r"this spot", "", query)
    tokens = query.split()
    tokens = [self.ps.stem(w) for w in tokens if w not in self.stopwords]
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    tokens = [self.ps.stem(w) for w in tokens if w not in self.stopwords]
    return " ".join(sorted(tokens))
  
  def delexicalize(self, domain, name, query):
    """
    Replace occurrence of entity in sentence by domain
    For example: domain = 'hotel', name = 'A and B Guest House'
    Can I bring my pet to A and B Guest House? -> Can I bring my pet to this hotel?
    """
    # TODO: Need Trie here for fast entity matching
    return query

  def __iter__(self):
    for q in self.kb_utterances_list:
      yield q

  def __getitem__(self, index):
    return self.kb_utterances_list[index]

  def __len__(self):
    return len(self.kb_utterances_list)

  def save(self, out_file):
    with open(out_file, 'w') as f:
      for q in self.kb_utterances_list:
        f.write("{}\n".format(q))

  def collate_fn(self, batch):
    """
    Received a batch of queries.
    Return: dictionary of inputs
    """
    output = self.tokenizer(batch, padding=True, add_special_tokens = True, return_tensors="pt")
    output_dict = {x: output[x].to(device) for x in output}
    output_dict["query"] = batch

    return output_dict

[nltk_data] Downloading package stopwords to
[nltk_data]     /content/drive/MyDrive/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from torch.utils.data import DataLoader, SequentialSampler

# Use Seeger's version of knowledge.json with delexicalized sentence templates
#kb_dataset = KnowledgeDataset(dataroot, 'knowledge.json', tokenizer)
kb_eval_dataset = KnowledgeDataset(dataroot_eval, 'kb_eval_wtemplates.json', tokenizer, title_field="title_template")

kb_eval_dataloader = DataLoader(
  kb_eval_dataset,
  sampler=SequentialSampler(kb_eval_dataset),
  batch_size=batch_size,
  collate_fn=kb_eval_dataset.collate_fn
)

In [ ]:
print(len(kb_eval_dataset.kb_utterances))
print(len(kb_eval_dataset.kb_utterances1))
print(len(kb_eval_dataset.kb_utterances2))
print(kb_eval_dataset.kb_utterances1)
print(kb_eval_dataset.kb_utterances2)

print(len(kb_eval_dataset.doc_clusters))
print(kb_eval_dataset.doc_clusters)

6608
4376
4715
{'children locat welcom': ['hotel_0_0', 'hotel_8_8', 'hotel_23_4'], 'bring hotel pet': ['hotel_0_1', 'hotel_110124_0', 'hotel_110202_3'], 'guest onsit park': ['hotel_0_2'], 'checkin time': ['hotel_0_3', 'hotel_12_11', 'hotel_13_2', 'hotel_14_18', 'hotel_21_20', 'hotel_32_2', 'hotel_110005_18', 'hotel_110006_21', 'hotel_110021_17', 'hotel_110024_17', 'hotel_110028_18', 'hotel_110043_18', 'hotel_110046_15', 'hotel_110090_15', 'hotel_110094_20', 'hotel_110118_19', 'hotel_110141_25', 'hotel_110147_14', 'hotel_110149_23', 'hotel_110157_21', 'hotel_110163_23'], 'allow properti smoke': ['hotel_0_4', 'hotel_110114_13'], 'languag spoken': ['hotel_0_5', 'hotel_11_3', 'hotel_12_6', 'hotel_21_17', 'hotel_23_2', 'hotel_32_4', 'hotel_110002_16', 'hotel_110018_3', 'hotel_110024_16', 'hotel_110035_22', 'hotel_110053_13', 'hotel_110055_16', 'hotel_110177_21', 'hotel_110182_20', 'hotel_110200_14'], 'make park reserv': ['hotel_0_6', 'hotel_3_8', 'hotel_6_9', 'hotel_11_2', 'hotel_110036_4',

In [ ]:
X = [x for x in kb_eval_dataset.doc_clusters if len(x) > 400]
print(len(X))

1


In [ ]:
for i in range(len(X)):
  input("Press Enter to continue...") 
  title_set = set({})
  body_set = set({})
  for kid in X[i]:
    title,body = kb_eval_dataset.get_doc(kid).split("\t")
    title_set.add(title)
    body_set.add(body)
  print(title_set)
  print(body_set)

Press Enter to continue...
{'Is there wifi available?', 'Do you charge for WiFi?', 'What is the charge for internet?', 'How much do you charge for WIFI?', 'Is Wifi available at this hotel?', 'Is there free WiFi available at this restaurant?', 'Does this restaurant have WiFi available?', 'Do you offer free WiFi?', 'Do you have wifi there?', 'Is the WiFi here free?', 'How much to get onto your wireless?', 'How much is the wifi?', 'Does this restaurant offer WiFi?', 'Is WiFi available at this hotel?', 'How much does it cost for WiFi there?', 'How much does it cost for the Wi-Fi?', 'Do you offer free wifi at your restaurant?', 'Do they offer wifi?', 'Is wifi offered?', 'Do you offer WI-FI?', 'Is Wifi available at your hotel?', 'Is Wifi available?', 'Do they offer free wifi?', 'Is there wifi at your restaurant?', 'Do they have wifi available?', 'Is the WiFi free at this hotel?', 'How much does it cost for WiFi?', 'Does the hotel have WiFi?', 'Is there WiFi available?', 'Does this hotel have

In [ ]:
X = [C for C in kb_eval_dataset.doc_clusters if len(C) > 200]
Y = [C for C in kb_eval_dataset.doc_clusters2 if len(C) > 200]

print(len(X))
print(len(Y))

for i in range(len(X)):
  for kid in X[i]:
    print(i, kid, kb_eval_dataset.get_doc(kid))
  print("\n\n")

for i in range(len(Y)):
  for kid in Y[i]:
    print(i, kid, kb_eval_dataset.get_doc(kid))
  print("\n\n")

13
0
0 hotel_110201_17 What time can I check in?	Check-in time at this hotel begins at 3 pm.
0 hotel_110169_11 What time can I check in?	check in time is at 3 pm
0 hotel_110008_18 What is the check-out time at this hotel?	The check-out time at this hotel is 10:00AM.
0 hotel_110061_13 What is the check in time at this hotel?	Check-in is front 3:00 p.m. to 11:00 p.m at this hotel.
0 hotel_2_36 When is check out time at this hotel?	Check out time is at 10:30 at this hotel.
0 hotel_110193_3 Check in and checkout time is when?	Check-in begins at 2:00 pm at this hotel.
0 hotel_110053_14 What time is check-in at this hotel?	this hotel check-in time starts at 3:00 PM.
0 hotel_110200_12 What is the check in time at the hotel?	Check-in time is at 3:00 PM.
0 hotel_0_8 what time do I check out?	Check out times range from 7:30 AM to 10:00 AM.
0 hotel_110076_24 When is check out time at this hotel?	Check out time is between 7 AM to 12 PM at this hotel.
0 hotel_110117_1 Check in and checkout time is 

In [ ]:
import pickle

# merged title clusters
print(len(kb_eval_dataset.doc_clusters))
print(sum([len(x) for x in kb_eval_dataset.doc_clusters]))
with open("kb_doc_clusters.pkl", "wb") as F:
  pickle.dump(kb_eval_dataset.doc_clusters, F)

# non-merged body clusters (could be noisy)
print(len(kb_eval_dataset.doc_clusters2))
print(sum([len(x) for x in kb_eval_dataset.doc_clusters2]))
with open("kb_doc_clusters2.pkl", "wb") as F:
  pickle.dump(kb_eval_dataset.doc_clusters2, F)

3937
12039
4715
12039


In [ ]:
!pwd

with open("kb_doc_clusters.pkl", "rb") as F:
  X = pickle.load(F)

X = [x for x in X if len(x) > 1]
print(len(X))
print(X[0])

/content/drive/My Drive/dstc9_bertemb
221
['hotel_0_26', 'hotel_14_6', 'hotel_2_10']


In [ ]:
i = 0
for batch in kb_eval_dataloader:
  i += 1
  if i >= 2:
    break
  print(batch)

{'input_ids': tensor([[  101,  2024,  2336, 10979,  2012,  2023,  3295,  1029,   102,     0,
             0,     0,     0,     0],
        [  101,  2064,  1045,  3288,  2026,  9004,  2000,  2023,  3309,  1029,
           102,     0,     0,     0],
        [  101,  2079,  2017,  2031,  2006, 28032,  2063,  5581,  2005,  2115,
          6368,  1029,   102,     0],
        [  101,  2054,  2051,  2003,  4638,  1011,  1999,  2045,  1029,   102,
             0,     0,     0,     0],
        [  101,  2003,  9422,  3039,  2006,  1996,  3200,  1029,   102,     0,
             0,     0,     0,     0],
        [  101,  2054,  4155,  2024,  5287,  1029,   102,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  2323,  1045,  2191,  1037, 11079,  2005,  5581,  1029,   102,
             0,     0,     0,     0],
        [  101,  2024,  2336,  3039,  2000,  4638,  1999,  2182,  1029,   102,
             0,     0,     0,     0],
        [  101,  2054,  2051,  2079,  1045,  4638,

## Dump embedding

In [ ]:
tokenizer.mask_token

'[MASK]'

In [ ]:
import torch
from torch import nn

a = torch.ones(32, 2, 10)  
m = nn.Conv1d(2, 2, 2)
m.weight.data.fill_(1.0)
m.bias.data.fill_(0.0)
out = m(a)
print(out.size())
#print(m)
print(out)

torch.Size([32, 2, 10])
tensor([[[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2.

In [ ]:
import torch
import numpy as np

# Calculate N-gram embedding based on bert hidden states
def Ngram_Embedding(model, tokenizer, text, N=2):
  # set model to evaluation mode
  model.eval()
  
  vecs = None
  with torch.no_grad():
    batch = tokenizer(text, add_special_tokens = True, return_tensors="pt", padding=True)
    batch.to(device)
    input_ids = batch["input_ids"]
    token_type_ids = batch["token_type_ids"]
    attention_mask = batch["attention_mask"]

    # multiply error_mask with attention_mask (i.e. error positions should not be considered in feed-forwarding)
    # TODO
    hidden_states, _ = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, return_dict=False)
  
    # Out: [B, H] after summing all vectors. Padding vectors will be zero
    masked_hidden_states = hidden_states * attention_mask[..., None]

    B = masked_hidden_states.shape(0)
    L = masked_hidden_states.shape(1)
    D = masked_hidden_states.shape(2)
  
    # copy the results from GPU memory into CPU memory
    if device == 'cpu':
      H = masked_hidden_states.numpy()
    else:
      H = masked_hidden_states.cpu().numpy()
  

# Get one sentence embedding (average hidden states)
# error positions will be converted into mask token id
def Embedding(model, tokenizer, text):
  # set model to evaluation mode
  model.eval()
  
  vecs = None
  with torch.no_grad():
    batch = tokenizer(text, add_special_tokens = True, return_tensors="pt", padding=True)
    batch.to(device)
    input_ids = batch["input_ids"]
    token_type_ids = batch["token_type_ids"]
    attention_mask = batch["attention_mask"]

    # multiply error_mask with attention_mask (i.e. error positions should not be considered in feed-forwarding)
    # TODO
  
    hidden_states, _ = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, return_dict=False)
  
    # Out: [B, H] after summing all vectors. Padding vectors will be zero
    #hidden_states_sum = torch.sum(hidden_states * attention_mask[..., None], axis=1)
  
    # Out: [B] after calculating the length
    #hidden_states_len = torch.sum(attention_mask, axis=1)
  
    # Out: [B, H], calculating the averaged vector
    #avg_emb = hidden_states_sum / hidden_states_len[:, None]

    # Use CLS token (the first token in 2nd dimension)
    avg_emb = hidden_states[:, 0]
  
    # copy the results from GPU memory into CPU memory
    if device == 'cpu':
      avg_emb_np = avg_emb.numpy()
    else:
      avg_emb_np = avg_emb.cpu().numpy()
  
    return np.copy(avg_emb_np)
  
def Mean_Embedding(model, dataloader):
  # set model to evaluation mode
  model.eval()
  
  vecs = None
  with torch.no_grad():
    for i, batch in enumerate(dataloader):
      #if i >= 2:
      #  break
      queries = batch["query"]
      input_ids = batch["input_ids"]
      token_type_ids = batch["token_type_ids"]
      attention_mask = batch["attention_mask"]
  
      hidden_states, _ = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, return_dict=False)
  
      # Out: [B, H] after summing all vectors. Padding vectors will be zero
      #hidden_states_sum = torch.sum(hidden_states * attention_mask[..., None], axis=1)
  
      # Out: [B] after calculating the length
      #hidden_states_len = torch.sum(attention_mask, axis=1)
  
      # Out: [B, H], calculating the averaged vector
      #avg_emb = hidden_states_sum / hidden_states_len[:, None]

      # Use CLS token (the first token in 2nd dimension)
      avg_emb = hidden_states[:, 0]
  
      # copy the results from GPU memory into CPU memory
      if device == 'cpu':
        avg_emb_np = avg_emb.numpy()
      else:
        avg_emb_np = avg_emb.cpu().numpy()
  
      if vecs is None:
        vecs = np.copy(avg_emb_np)
      else:
        vecs = np.concatenate((vecs, avg_emb_np), axis=0)
       
  return vecs

In [ ]:
train_vecs = Mean_Embedding(model, train_dataloader)
val_vecs = Mean_Embedding(model, valid_dataloader)
test_vecs = Mean_Embedding(model, test_dataloader)

In [ ]:
kb_eval_dataset.entity_utterances["hotel"][0]

['Are children welcomed at this location?',
 'Can I bring my pet to this hotel?',
 'Do you have onsite parking for your guests?',
 'What time is check-in there?',
 'Is smoking allowed on the property?',
 'What languages are spoken?',
 'Should I make a reservation for parking?',
 'Are children allowed to check in here?',
 'what time do I check out?',
 'Can my small dog stay with me?',
 'What type of credit cards do you take at your location?',
 'Is there wifi available?',
 'Is there a charge for parking?',
 'How much is the wifi?',
 'Is there housekeeping services?',
 'Is there parking available?',
 'Can we have extra beds put into the room?',
 'Do this property have a meeting or banquet facility?',
 'Is there free breakfast at this hotel?',
 'What is on the breakfast menu?',
 'Is smoking allowed anywhere in the property?',
 'Are the rooms at this hotel accessible to those with disabilities?',
 'Does this hotel offer accessible parking?',
 'Does this site offer concierge service?',
 'Is

In [ ]:
kb_eval_dataset.entity_utterances["hotel"][1]

['Are pets welcomed at this property?',
 'Do you have parking available for my car?',
 'What are your breakfast options?',
 'What age does a child need to be to stay there?',
 'Is the hotel WiFi accessible?',
 'Do you serve a free breakfast?',
 'Will I be able to request additional beds there?',
 'Is there any smoking section at this property?',
 'Is there wifi available, and is it an extra charge?',
 'What time is check in for this hotel?',
 'When is the latest I can check out?',
 'What type of credit and debit cards do you take?',
 'What languages does your staff speak?',
 'Are there facility for people with disability?',
 'Is there accessible parking available?',
 'Will there be a concierge service at this hotel?',
 'Does this hotel have a gym?',
 'Will I be able to use a kitchen when I stay here?',
 'Do you provide laundry services?',
 'Do you have a lift for guests?',
 'Are there luggage storage service?',
 'Does this site have facility to hold banquet or meeting?',
 'What is the 

In [ ]:
def norm2(X):
  # normalize vectors with L2
  Z = np.linalg.norm(X, axis=-1, keepdims=True)
  return X / Z

v0 = Embedding(model, tokenizer, kb_eval_dataset.entity_utterances["hotel"][0])
v1 = Embedding(model, tokenizer, kb_eval_dataset.entity_utterances["hotel"][1])

v0 = norm2(v0)
v1 = norm2(v1)

In [ ]:
print(v0.shape)
print(v1.shape)
D = np.matmul(v0, v1.T)
J = np.argmax(D, axis=-1)

print(J.shape)

for i in range(v0.shape[0]):
  sent0 = kb_eval_dataset.entity_utterances["hotel"][0][i]
  sent1 = kb_eval_dataset.entity_utterances["hotel"][1][J[i]]
  print(sent0, sent1, D[i][J[i]])

(38, 768)
(37, 768)
(38,)
Are children welcomed at this location? Are pets welcomed at this property? 0.98012155
Can I bring my pet to this hotel? Are pets allowed at this hotel? 0.95922995
Do you have onsite parking for your guests? Do you have a lift for guests? 0.9786444
What time is check-in there? What time is check in for this hotel? 0.975145
Is smoking allowed on the property? Is there any smoking section at this property? 0.9646862
What languages are spoken? What languages does your staff speak? 0.9556414
Should I make a reservation for parking? Do I need a parking reservation? 0.98534316
Are children allowed to check in here? Are pets welcomed at this property? 0.9602705
what time do I check out? When is the latest I can check out? 0.9742757
Can my small dog stay with me? Will I be able to use a kitchen when I stay here? 0.95514494
What type of credit cards do you take at your location? What type of credit and debit cards do you take? 0.9809994
Is there wifi available? Is ther

In [ ]:
kb_eval_dataset.entity_utterances["hotel"][1][30]

'Are pets allowed at this hotel?'

In [ ]:
kb_eval_vecs = Mean_Embedding(model, kb_eval_dataloader)

KeyboardInterrupt: ignored

In [ ]:
#print(train_vecs.shape)
#print(val_vecs.shape)
#print(test_vecs.shape)
#print(kb_vecs.shape)
print(kb_eval_vecs.shape)

(6608, 768)


In [ ]:
import pickle

# dump the object
with open(os.path.join(dataroot, 'kb.pkl'), "wb") as F:
  pickle.dump(kb_eval_dataset, F)

In [ ]:
with open(os.path.join(dataroot, 'kb.pkl'), "rb") as F:
  rep = pickle.load(F)

print(rep[103])

What is the minimum age required to check into this hotel?


In [ ]:
train_dataset.save(os.path.join(dataroot, 'train', 'train_query.txt'))
valid_dataset.save(os.path.join(dataroot, 'val', 'valid_query.txt'))
kb_dataset.save(os.path.join(dataroot, 'kb_query.txt'))

test_dataset.save(os.path.join(dataroot_eval, 'test', 'test_query.txt'))
kb_eval_dataset.save(os.path.join(dataroot_eval, 'kb_eval_query.txt'))

In [ ]:
print(dataroot_eval)

alexa-with-dstc9-track1-dataset/data_eval


In [ ]:
#np.save(os.path.join(dataroot, 'train', 'train_vecs.npy'), train_vecs)
#np.save(os.path.join(dataroot, 'val', 'val_vecs.npy'), val_vecs)
#np.save(os.path.join(dataroot, 'kb_vecs.npy'), kb_vecs)

#np.save(os.path.join(dataroot_eval, 'test', 'test_vecs.npy'), test_vecs)
np.save(os.path.join(dataroot_eval, 'kb_eval_vecs.npy'), kb_eval_vecs)

In [ ]:
!pwd
!ls -lrt alexa-with-dstc9-track1-dataset/data

/content/drive/My Drive/dstc9_bertemb
total 10945
drwx------ 2 root root    4096 Jun 10 13:24 train
-rw------- 1 root root    3280 Jun 10 13:24 README.md
-rw------- 1 root root    1372 Jun 10 13:24 output_schema.json
-rw------- 1 root root  471645 Jun 10 13:24 knowledge.json
drwx------ 2 root root    4096 Jun 10 13:24 val
-rw------- 1 root root 1060527 Jun 15 13:38 kb_wtemplates.json
-rw------- 1 root root  268449 Jun 15 14:18 kb_eval_query.txt
-rw------- 1 root root   73539 Jun 15 14:18 kb_query.txt
-rw------- 1 root root 5594240 Jun 15 14:19 kb_vecs.npy
-rw------- 1 root root 3723420 Jul  8 13:28 kb.pkl


In [ ]:
# Shape should be [B, H], same as train_vecs.npy
#train_vecs1 = np.load("alexa-with-dstc9-track1-dataset/data/train/train_vecs.npy")
#print(train_vecs1.shape)
#print(train_vecs1[:2])

kb_vecs1 = np.load("alexa-with-dstc9-track1-dataset/data_eval/kb_eval_vecs.npy")
#print(kb_eval_vecs[:2])
print(kb_vecs1[:2])
print(kb_vecs1.shape)

[[-0.06437036  0.0855116  -0.12253223 ... -0.3307651   0.28563878
   0.40609637]
 [-0.05256403  0.0221472   0.24282537 ... -0.45217702  0.28523088
   0.5983836 ]]
(6608, 768)


In [ ]:
# normalize vectors with L2
x = np.array([[1,2], [2,2], [1,1]], dtype=np.float)
z = np.linalg.norm(x, axis=-1, keepdims=True)
print(z)
norm1 = x / z
norm1

[[2.23606798]
 [2.82842712]
 [1.41421356]]


array([[0.4472136 , 0.89442719],
       [0.70710678, 0.70710678],
       [0.70710678, 0.70710678]])

In [ ]:
def norm2(X):
  # normalize vectors with L2
  Z = np.linalg.norm(X, axis=-1, keepdims=True)
  return X / Z

kb_vecs1 = norm2(kb_vecs1)
print(kb_vecs1[:2])

[[-0.00442606  0.00587972 -0.00842523 ... -0.02274317  0.01964032
   0.0279229 ]
 [-0.00360656  0.00151958  0.01666091 ... -0.0310251   0.01957047
   0.04105673]]


In [ ]:
score = np.matmul(train_vecs1, kb_vecs1.T)
print(score.shape)

(64250, 1821)


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
syns = wordnet.synsets("tv")
print(syns)

[nltk_data] Downloading package wordnet to
[nltk_data]     /content/drive/MyDrive/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[Synset('television.n.01'), Synset('television_receiver.n.01')]


In [ ]:
import sys

topK = 10

# try the demo
while True:
  raw_text = input(">>>")
  while not raw_text:
    print('Prompt should not be empty!')
    raw_text = input(">>> ")
  # find the nearest sentence in KB
  v = Embedding(model, tokenizer, [raw_text.strip()])
  #print(v.shape)
  #print(kb_vecs1.shape)
  # shape of D: (1, Number of Title in KB)
  D = np.matmul(norm2(v), kb_vecs1.T)
  #print(D.shape)
  kv = {}
  for i in range(kb_vecs1.shape[0]):
    kv[i] = D[0][i]
  k = 0
  for j, v in sorted(kv.items(), key=lambda item: item[1], reverse=True):
    if k < topK:
      print(kb_eval_dataset[j], v)
    k += 1

>>>do you have free wifi?
Do you have free WiFi? 1.0
Do you have free wifi? 1.0
DO you have free wifi? 1.0
Do you have free WIFI? 1.0
Do you have free Wifi? 1.0
Do you have free wifi there? 0.9917401
Do you have free WiFi there? 0.9917401
Do you have WiFi? 0.99147046
Do you have wifi? 0.99147046
Do you have Wifi? 0.99147034
>>>do you have wif i?
DO you have BYOB? 0.9721108
Do you have BYOB? 0.97211075
Do you do BYOB? 0.9672195
Do you allow BYOB? 0.9613637
Do you take Apple Pay? 0.96111476
Do you take apple pay? 0.96111476
Do you have hatal options? 0.95816916
Do you have banquest facilities? 0.9579008
Are you kid friendly? 0.95649487
Do you take reseravtions? 0.95607376
>>>do you ive wifi?
Do you ahve take out? 0.94344133
Do you guys have gratis wifi? 0.9359836
Do you have WiFi? 0.9357191
Do you have wifi? 0.9357191
Do you have Wifi? 0.935719
Do you guys have WiFi? 0.934245
Do yall have a dress code? 0.9334806
DO you have BYOB? 0.93025154
Do you have BYOB? 0.9302514
Do yo have free wif

Next step: 

_1_: Use Faiss to index kb_vecs.npy

_2_: For each query in dialogue, use Faiss to retrieve K-nearest neighbors of the query. These delexicalized KB queries will be used to create negative samples for KB selection.

# Faiss

Faiss creates index of embedding vectors. During test time, given an embedding vector, Faiss locates the K-nearest neighbor (KNN) of the vector.

First, we use Faiss to create index of the KB vectors. 

Then, for each dialogue query, we retrieve the KNN of it. These KNN candidates will be used for further processing.

Faiss: https://github.com/facebookresearch/faiss

In [ ]:
!git clone https://github.com/facebookresearch/faiss.git

Cloning into 'faiss'...
remote: Enumerating objects: 18946, done.
remote: Counting objects: 100% (2439/2439), done.
remote: Compressing objects: 100% (377/377), done.
remote: Total 18946 (delta 2140), reused 2318 (delta 2062), pack-reused 16507
Receiving objects: 100% (18946/18946), 37.51 MiB | 13.35 MiB/s, done.
Resolving deltas: 100% (16311/16311), done.
Checking out files: 100% (557/557), done.


In [ ]:
%cd faiss/
!ls

/content/drive/My Drive/dstc9_bertemb/faiss
benchs	      CMakeLists.txt	  CONTRIBUTING.md  faiss       README.md
c_api	      CODE_OF_CONDUCT.md  demos		   INSTALL.md  tests
CHANGELOG.md  conda		  Dockerfile	   LICENSE     tutorial
cmake	      contrib		  Doxyfile	   misc
